In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from brian2 import *
from brian2tools import *
from scipy.stats import skew
import numpy as np
import pandas as pd



# Customize matplotlib
matplotlib.rcParams.update(
    {
        'text.usetex': False,
        'font.family': 'stixgeneral',
        'mathtext.fontset': 'stix',
        'font.size': 22,
        'ytick.color' : 'black',
        "xtick.color" : 'black',
        'axes.labelcolor' : 'black',
        'axes.edgecolor' : 'black'
	}
)

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [2]:
n_points = 50
n_samples = 1
vec_tauA = np.linspace(0.1, 100, n_points)

EL = -60*mV
EE = 0*mV
EI = -80*mV
tauI = 10*ms
tauL = 20*ms

eqs = '''
dv/dt = ((EL - v) + gA*(EE - v) + gI*(EI - v))/tauL : volt
dgA/dt = -gA/tauA : 1
dgI/dt = -gI/tauI : 1
tauA : second
'''

wI_list = [0.1, 1, 10]
mean_V_ta = {'tauA':vec_tauA, 0.1:np.zeros(n_points*n_samples), 1:np.zeros(n_points*n_samples), 10:np.zeros(n_points*n_samples)}
var_V_ta = {'tauA':vec_tauA, 0.1:np.zeros(n_points*n_samples), 1:np.zeros(n_points*n_samples), 10:np.zeros(n_points*n_samples)}
mean_gA_ta = {'tauA':vec_tauA, 0.1:np.zeros(n_points*n_samples), 1:np.zeros(n_points*n_samples), 10:np.zeros(n_points*n_samples)}
var_gA_ta = {'tauA':vec_tauA, 0.1:np.zeros(n_points*n_samples), 1:np.zeros(n_points*n_samples), 10:np.zeros(n_points*n_samples)}
skew_gA_ta = {'tauA':vec_tauA, 0.1:np.zeros(n_points*n_samples), 1:np.zeros(n_points*n_samples), 10:np.zeros(n_points*n_samples)}

for wI in wI_list:
	print(wI, end='\t')
	for i, tauA_value in enumerate(vec_tauA):
		start_scope()
		print(i/size(vec_tauA), end=' ')
		G = NeuronGroup(1, eqs, method='rk4')
		GPI = PoissonGroup(100, rates=5*Hz)
		GPE = PoissonGroup(400, rates=5*Hz)

		SE = Synapses(GPE, G, 'w : 1', on_pre='gA += w')
		SI = Synapses(GPI, G, 'w : 1', on_pre='gI += w')
		SE.connect()
		SI.connect()

		G.v = EL
		SE.w = 0.8
		SI.w = wI

		G.tauA = tauA_value*ms
		
		run(5000*ms)

		M = StateMonitor(G, ('v', 'gA'), record=True)

		run(20000*ms)
		mean_V_ta[wI][i] = np.mean(M.v/mV)
		var_V_ta[wI][i] = np.var(M.v/mV)
		mean_gA_ta[wI][i] = np.mean(M.gA/mV)
		var_gA_ta[wI][i] = np.var(M.gA/mV)
		skew_gA_ta[wI][i] = skew(np.squeeze(M.gA/mV))

	print()

0.1	0.0 0.02 0.04 0.06 0.08 0.1 0.12 0.14 0.16 0.18 0.2 0.22 0.24 0.26 0.28 0.3 0.32 0.34 0.36 0.38 0.4 0.42 0.44 0.46 0.48 0.5 0.52 0.54 0.56 0.58 0.6 0.62 0.64 0.66 0.68 0.7 0.72 0.74 0.76 0.78 0.8 0.82 0.84 0.86 0.88 0.9 0.92 0.94 0.96 0.98 
1	0.0 0.02 0.04 0.06 0.08 0.1 0.12 0.14 0.16 0.18 0.2 0.22 0.24 0.26 0.28 0.3 0.32 0.34 0.36 0.38 0.4 0.42 0.44 0.46 0.48 0.5 0.52 0.54 0.56 0.58 0.6 0.62 0.64 0.66 0.68 0.7 0.72 0.74 0.76 0.78 0.8 0.82 0.84 0.86 0.88 0.9 0.92 0.94 0.96 0.98 
10	0.0 0.02 0.04 0.06 0.08 0.1 0.12 0.14 0.16 0.18 0.2 0.22 0.24 0.26 0.28 0.3 0.32 0.34 0.36 0.38 0.4 0.42 0.44 0.46 0.48 0.5 0.52 0.54 0.56 0.58 0.6 0.62 0.64 0.66 0.68 0.7 0.72 0.74 0.76 0.78 0.8 0.82 0.84 0.86 0.88 0.9 0.92 0.94 0.96 0.98 


In [3]:
data_directory_path = "../../Data/V_data/"

# Convert to pandas DataFrame
df_mean_gA_ta = pd.DataFrame.from_dict(mean_gA_ta)
df_var_gA_ta = pd.DataFrame.from_dict(var_gA_ta)
df_skew_gA_ta = pd.DataFrame.from_dict(skew_gA_ta)
df_mean_V_ta = pd.DataFrame.from_dict(mean_V_ta)
df_var_V_ta = pd.DataFrame.from_dict(var_V_ta)


# Save Dataframe do csv file
df_mean_gA_ta.to_csv(f"{data_directory_path}mean_gA_wI.dat")
df_var_gA_ta.to_csv(f"{data_directory_path}var_gA_wI.dat")
df_skew_gA_ta.to_csv(f"{data_directory_path}skew_gA_wI.dat")
df_mean_V_ta.to_csv(f"{data_directory_path}mean_V_wI.dat")
df_var_V_ta.to_csv(f"{data_directory_path}var_V_wI.dat")


In [17]:
df_r = pd.read_csv(f"{data_directory_path}mean_V_ta_iRate.dat", index_col=0)
df_r

,tauA,5,10,20
0,0.10000,-72.627699,-75.300198,-77.049282
1,0.60201,-70.214860,-72.425135,-73.766838
2,1.10402,-68.229024,-69.665390,-70.671601
3,1.60603,-65.982644,-67.172607,-67.958467
4,2.10804,-64.115257,-64.808545,-65.367030
...,...,...,...,...
195,97.99196,-9.773240,-8.510439,-7.978084
196,98.49397,-9.716927,-8.505683,-7.955067
197,98.99598,-9.621671,-8.456356,-7.910107
198,99.49799,-9.589893,-8.448515,-7.897734
